# Reactor Kinetics Example 

Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a reactor kinetics experiment with the Pyomo.DOE.
    

In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
import pandas as pd 
from itertools import permutations, product, combinations
import idaes

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp

## Define Reaction Example Mathematical Model

Consider two chemical reactions that converts molecule $A$ to desired product $B$ and a less valuable side-product $C$.

$A \overset{k_1}{\rightarrow} B \overset{k_2}{\rightarrow} C$

Our ultimate goals is to design a large-scale continous reactor that maximizes the production of $B$. This general sequential reactions problem is widely applicable to CO$_2$ capture and industry more broadly (petrochemicals, pharmasuticals, etc.).

The rate laws for these two chemical reactions are:

$r_A = -k_1 C_A$

$r_B = k_1 C_A - k_2 C_B$

$r_C = k_2 C_B$

Here, $C_A$, $C_B$, and $C_C$ are the concentrations of each species. The rate constants $k_1$ and $k_2$ depend on temperature as follows:

$k_1 = A_1 \exp{\frac{-E_1}{R T}}$

$k_2 = A_2 \exp{\frac{-E_2}{R T}}$

$A_1, A_2, E_1$, and $E_2$ are fitted model parameters. $R$ is the ideal-gas constant and $T$ is absolute temperature.

Using the **CCSI$^2$ toolset**, we would like do the following perform:

Perform **uncertainty quantification** and **design of experiments** on a small-scale **batch reactor** to infer parameters $A_1$, $A_2$, $E_1$, and $E_2$.

### Batch reactor

The concenrations in a batch reactor evolve with time per the following differential equations:

$$ \frac{d C_A}{dt} = r_A = -k_1 C_A $$

$$ \frac{d C_B}{dt} = r_B = k_1 C_A - k_2 C_B $$

$$ \frac{d C_C}{dt} = r_C = k_2 C_B $$

This is a linear system of differential equations. Assuming the feed is only species $A$, i.e., 

$$C_A(t=0) = C_{A0} \quad C_B(t=0) = 0 \quad C_C(t=0) = 0$$

When the temperature is constant, it leads to the following analytic solution:

$$C_A(t) = C_{A,0} \exp(-k_1 t)$$

$$C_B(t) = \frac{k_1}{k_2 - k_1} C_{A,0} \left[\exp(-k_1 t) - \exp(-k_2 t) \right]$$

$$C_C(t) = C_{A,0} - \frac{k_2}{k_2 - k_1} C_{A,0} \exp(-k_1 t) + \frac{k_1}{k_2 - k_1} \exp(-k_2 t) C_{A,0} = C_{A,0} - C_{A}(t) - C_{B}(t)$$

In [2]:
#from reactor_models import *
from reactor_model_multipleversions import *

Three versions of this model is accomplished: 

Dynamic-DAE model: Temperature varying model discretized and integrated by Pyomo.DAE

Constant-DAE model: Temperature constant model discretized and integrated by Pyomo.DAE

Constant-analytical model: Temperature constant model discretized manually and using the analytical expressions for state variables.

In [3]:
# choose model from 'dynamic-DAE', 'constant-DAE', 'constant-analytical'
model_opt = 'dynamic-DAE-measure'
#model_opt = 'dynamic-DAE'
#model_opt = 'constant-DAE'
#model_opt = 'constant-analytical'

#createmod = create_model_overall
if (model_opt == 'dynamic-DAE-measure'):
    createmod = create_model_dae_measure
    args_ = [True, False, False]
    disc = disc_for_measure
    t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]

elif (model_opt=='dynamic-DAE'):
    createmod = create_model_dae
    args_ = [True, False, False]
    disc = discretizer
    t_control = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
    
elif (model_opt=='constant-DAE'):
    createmod = create_model_dae_const
    args_ = [True, True, False]
    disc = discretizer
    t_control = [0]
    
elif (model_opt=='constant-analytical'):
    createmod = create_model_alge
    args_ = [False, True, False]
    disc = None
    t_control = [0]

    
# design variable and its control time set
dv_pass = {'CA0': [0],'T': t_control}
    

# Define measurement time points
t_measure = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
t_measure_ca = [0, 0.25, 0.5, 0.75, 1]
t_measure_cb = [0, 0.125, 0.375, 0.625, 0.875, 1]
measure_pass = {'CA': t_measure, 'CB': t_measure, 'CC': t_measure}
measure_class =  Measurements(measure_pass)
#measure_pass = {'C':{"'CA'": t_measure, "'CB'": t_measure, "'CC'": t_measure}}
#measure_pass = {'C':{'CA': t_measure, 'CB': t_measure, 'CC': t_measure}}
#measure_pass = {'C':{'CA': t_measure_ca, 'CB': t_measure_cb, 'CC': t_measure}}

measure_subset = {'CA': t_measure_ca, 'CB': t_measure_cb, 'CC': t_measure}
measure_subclass = Measurements(measure_subset)
'''
comb = [t_measure, t_measure_ca, t_measure_cb]
#unique = list(set(t_measure_ca+t_measure_cb))
d = []
for i in comb:
    d += i
    unique = list(set((d)))

print(unique)
'''

All measurements are flattened.
Flatten measurement name: ['CA', 'CB', 'CC']
Flatten measurement variance: {'CA': 1, 'CB': 1, 'CC': 1}
All measurements are flattened.
Flatten measurement name: ['CA', 'CB', 'CC']
Flatten measurement variance: {'CA': 1, 'CB': 1, 'CC': 1}


'\ncomb = [t_measure, t_measure_ca, t_measure_cb]\n#unique = list(set(t_measure_ca+t_measure_cb))\nd = []\nfor i in comb:\n    d += i\n    unique = list(set((d)))\n\nprint(unique)\n'

In [4]:
# Define parameter nominal value 
parameter_dict = {'A1': 84.79085853498033, 'A2': 371.71773413976416, 'E1': 7.777032028026428, 'E2': 15.047135137500822}

def generate_exp(t_set, CA0, T):  
    '''Generate experiments. 
    t_set: time control set for T.
    CA0: CA0 value
    T: A list of T 
    '''
    assert(len(t_set)==len(T)), 'T should have the same length as t_set'
    
    T_con_initial = {}
    for t, tim in enumerate(t_set):
        T_con_initial[tim] = T[t]
        
    dv_dict_overall = {'CA0': {0: CA0},'T': T_con_initial}
    return dv_dict_overall

In [6]:
# empty prior
prior_all = np.zeros((4,4))


# add prior information
prior_5_300 = pd.read_csv('fim_5_300_scale.csv')
prior_5_300_500 = pd.read_csv('fim_5_300_500_scale.csv')

print(prior_5_300)

#print(prior_5_300_500)

#prior_all = prior_5_300

prior_300 = np.asarray(prior_5_300)
prior_300_500 = np.asarray(prior_5_300_500)
prior_500 = prior_300_500 - prior_300

def newyear(ma):
    print('det:', np.linalg.det(ma))
    print('trace:', np.trace(ma))
    print('eig:', np.linalg.eigvals(ma))
    print('eig:', np.linalg.eigh(ma)[1])
    print('cond:', np.linalg.cond(ma))
    
#newyear(prior_300)

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

          A1          A2          E1          E2
0  22.529430    1.840343  -70.232733  -11.094330
1   1.840343   18.098481   -5.735650 -109.158661
2 -70.232733   -5.735650  218.941928   34.576808
3 -11.094330 -109.158661   34.576808  658.376446
The prior information FIM: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0. 0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


## Compute FIM 

This method computes an MBDoE optimization problem with no Degree of Freedom.

In [7]:
# choose from 'simultaneous', 'sequential', 'sipopt'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

if sensi_opt == 'direct_kaug':
    args_[2] = True
else:
    args_[2] = False
    

# Define experiments
if (model_opt=='dynamic-DAE') or (model_opt=='dynamic-DAE-measure'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [300])

print('Design variable:', exp1)

Design variable: {'CA0': {0: 5}, 'T': {0: 570.21, 0.125: 300, 0.25: 300, 0.375: 300, 0.5: 300, 0.625: 300, 0.75: 300, 0.875: 300, 1: 300}}


In [8]:
scena1 = {'scena-name': [0], 'A1':84.79085853498033, 'A2':371.71773413976416,
         'E1': 7.777032028026428, 'E2': 15.04713513750}

kitty = create_model_overall(scena1, [True, True, False])

discretizer(kitty)

kitty.CA0.fix()
for t in t_control:
    kitty.T[t].fix()

#kitty.pprint()
solver = SolverFactory('ipopt')
solver.solve(kitty, tee=True)

for t in t_control:
    try:
        print(value(kitty.CA[0, t]))
    except:
        print('no')
        
print(value(kitty.CA[0,0.25]))

NameError: name 'create_model_overall' is not defined

In [ ]:
from reactor_model_multipleversions import *

scena1 = {'scena-name': [0], 'A1':84.79085853498033, 'A2':371.71773413976416,
         'E1': 7.777032028026428, 'E2': 15.04713513750}

maomao = create_model_dae_measure(scena1)

#discretizer(maomao)
disc_for_measure(maomao)

maomao.CA0.fix()
for t in t_control:
    maomao.T[t].fix()
    
for z in maomao.scena:
    for t in maomao.t:
        maomao.dCdt_rule[z,'CC',t].deactivate()

    
solver = SolverFactory('ipopt')
solver.solve(maomao, tee=True)

time_set = []
for t in maomao.t:
    time_set.append(t)

for t in t_control:
    try:
        print(value(maomao.C[0,'CA', t]))
        #print(value(maomao.CC[0,t]))

    except:
        print('no')

In [9]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=disc, args=None)


if_s = True

result = doe_object.compute_FIM(exp1,mode=sensi_opt, FIM_store_name = 'dynamic.csv', 
                                store_output = 'store_output', read_output=None,
                                scale_nominal_param_value=if_s, formula='central')


result.calculate_FIM(doe_object.design_values)


Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'A1': {0: 84.8756493935153}, 'A2': {0: 371.71773413976416}, 'E1': {0: 7.777032028026428}, 'E2': {0: 15.047135137500822}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.0848756493935153, 'A2': 0.37171773413976417, 'E1': 0.007777032028026428, 'E2': 0.015047135137500823}, 'scena-name': [0]}
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Fram

AttributeError: 'ConcreteModel' object has no attribute 'CA'

In [ ]:
measure_subset = {'CA': t_measure, 'CC': t_measure}

measure_subclass = Measurements(measure_subset)

sub_result = result.subset(measure_subclass)

sub_result.calculate_FIM(doe_object.design_values)


In [ ]:
split = {'A1': [7.985123673392991e-08, 1.2160454865295378e-06, 2.3501485202359618e-06, 1.207393296454029e-06, 5.6706639384174196e-08, 4.4566128565293184e-08, 3.675770798849953e-08, 3.242695001404172e-08, 3.0965008335215316e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'A2': [6.407674391084583e-08, 2.98443048052377e-07, 5.356515231369485e-07, 7.755001085740786e-07, 1.0180478682286775e-06, 1.2635692492324324e-06, 1.5125545260730178e-06, 1.7655197304122794e-06, 1.9994770283915386e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'E1': [3.644640145239464e-08, 4.055196178853748e-07, 7.746030483701816e-07, 1.1413190392772774e-06, 2.8874680424451068e-08, 2.2612134387145488e-08, 1.8547385849387862e-08, 1.640954039316966e-08, 1.583444486641383e-08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'E2': [5.098567346095705e-06, 7.402483959140227e-06, 9.706586201474465e-06, 9.736789152725576e-06, 9.757012975342148e-06, 9.770408038178857e-06, 9.778790666103987e-06, 9.783361232251764e-06, 9.784806298540614e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

print(np.shape(split['A1']))

In [ ]:
FIM_subclass = FIM_result(['A1', 'A2', 'E1', 'E2'], measure_subclass, prior_FIM=prior_pass)
FIM_subclass.calculate_FIM(doe_object.jac, doe_object.design_values, jaco_involved=)


In [ ]:
print(doe_object.dsdp)

In [ ]:
dd = 'dd'
print(type(dd))

if type(dd) is str:
    print(dd)

In [ ]:
dd = '"' + dd + '"'
print(dd)

In [ ]:
print(doe_object.jac)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

### Run sequential DOE automatically

This method runs a series of experiments in a row.

In [ ]:
# this only works for constant T mode. 
exp1 = generate_exp(t_control, 5, [300])
exp2 = generate_exp(t_control, 5, [500.92])
exp3 = generate_exp(t_control, 5, [528])
exp4 = generate_exp(t_control, 5, [546.8])
exp5 = generate_exp(t_control, 5, [562])

In [ ]:

exp_all = [exp1, exp2, exp3, exp4, exp5]


doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 ['CA','CB','CC'], t_measure, createmod,
                                prior_FIM=prior_pass, discretize_model=disc)

# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
if_s = True
result = doe_object.sequential_exp(exp_all, mode=sensi_opt, tee_option=True,
                                    scale_nominal_param_value=if_s, formula='central')

In [ ]:
print('=======Result summary=======')
for i in range(len(exp_all)):
    print('The ', i+1,'th experiment has optimality log10() values:')
    print('A-optimality:', np.log10(result[0][i].trace))
    print('D-optimality:', np.log10(result[0][i].det))
    print('E-optimality:', np.log10(result[0][i].min_eig))
    print('Modified E-optimality:', np.log10(result[0][i].cond))

## Optimization

In [ ]:
# Define experiments
if (model_opt=='dynamic-DAE') or (model_opt=='dynamic-DAE-measure'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [300])

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 measure_pass, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=None)


In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Grid search for 2 design variables

In [ ]:
design_ranges = [list(np.linspace(1,5,9)), list(np.linspace(300,700,9))]

dv_apply_name = ['CA0','T']

dv_apply_time = [[0],t_control]

# Define experiments
if (model_opt=='dynamic-DAE'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [500])
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'

sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

#if sensi_opt == 'direct_kaug':
#    args_[2] = True

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 ['CA','CB','CC'], t_measure, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc, args=None)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, 
                                    scale_nominal_param_value=True
                                    )

### 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

## draw 1D sensitivity curve 

fixed = {"'CA0'": 5.0}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )



## Heatmap

In [ ]:
fixed = {}
all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )

## Grid search for 3 design variables

In [ ]:

#design_ranges = [list(np.linspace(1,5,2)),  list(np.linspace(300,700,2)), [300,500]]
design_ranges = [list(np.linspace(1,5,2)),  list(np.linspace(300,700,2)), [300,500]]

dv_apply_name = ['CA0', 'T', 'T']
dv_apply_time = [[0], [0], [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875,1]]

# Define experiments
if (model_opt=='dynamic-DAE'):
    exp1 = generate_exp(t_control, 5, [570.21, 300, 300, 300, 300, 300, 300, 300, 300])
else: 
    exp1 = generate_exp(t_control, 5, [500])

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 ['CA','CB','CC'], t_measure, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode='sequential_finite', 
                                    scale_nominal_param_value=True
                                    )

### Draw 1D sensitivity curve

In [ ]:
test = all_fim.extract_criteria()

In [ ]:
## draw 1D sensitivity curve 

fixed = {"'CA0'": 1.0, "'T2'": 300}

all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )

### Draw 2D sensitivity curve

In [ ]:
fixed = {"'T2'": 300}

all_fim.figure_drawing(fixed, ['CA0','T'], 'Reactor case','$C_{A0}$ [M]', 'T [K]' )

## Dynamic heatmap

In [ ]:
design_ranges = [list(np.linspace(300,700,18))]

dv_apply_name = ['T']

count = 0
for ct in t_control:
    print('Time controlled:', ct)
    dv_apply_time = [[ct]]

    # Define experiments
    exp1 = generate_exp(t_control, 5, [300, 300, 300, 300, 300, 300, 300, 300, 300])
    
    doe_object = DesignOfExperiments(parameter_dict, dv_pass, 
                                 ['CA','CB','CC'], t_measure, createmod,
                                 prior_FIM=prior_pass, discretize_model=disc)

    all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode='sequential_finite', 
                                    scale_nominal_param_value=True,
                                     filename = '300base_'+str(count)+'.csv'
                                    )
    
    test = all_fim.extract_criteria()

    ## draw 1D sensitivity curve 

    fixed = {}

    all_fim.figure_drawing(fixed, ['T'], 'Reactor case','T [K]','$C_{A0}$ [M]' )
    count += 1 

In [ ]:
filename_list = []
count = 0
for ct in t_control:
    print('Time controlled:', ct)
    filename_list.append('300base_'+str(count)+'.csv')
    count += 1
    
print(filename_list)

data_list = []
for i in filename_list:
    data_list.append(pd.read_csv(i))
print(np.shape(data_list))

In [ ]:
print(data_list[3]['D'].iloc[4])

In [ ]:
d_result = np.zeros((18,8))

for i in range(8):
    for j in range(18):
        d_result[j,i] = np.log10(data_list[i]['A'].iloc[j])
        
print(d_result)

In [ ]:
x_range1 = [0.125,0.25,0.375,0.5,0.625,0.75,0.875,1]
#x_range1 = [0.13, 0.25,0.38,0.5,0.63,0.75,0.88,1]
#x_range1 = [7.5, 15, 22.5, 30, 37.5, 45, 52.5, 60]
y_range1 = yLabel = [300, 321, 342, 363, 384, 405, 426, 447, 468, 489, 511, 532, 553, 574, 595, 616, 637, 658, 679, 700.0]

heatmap(x_range1, y_range1, 'Time [$min$]', 'Temperature [$K$]')

In [ ]:
import matplotlib.ticker as ticker
def heatmap(x_range, y_range, xlabel_text, ylabel_text, font_axes=16, font_tick=12, log_scale=True):
    '''
    Draw 2D heatmaps for all design criteria

    Parameters:
    ----------
    title_text: name of the figure, a string
    xlabel_text: x label title, a string.
        In a 2D heatmap, it should be the second design varialbe in the design_ranges
    ylabel_text: y label title, a string.
        In a 2D heatmap, it should be the first design variable in the dv_ranges
    font_axes: axes label font size
    font_tick: tick label font size
    log_scale: if True, the result matrix will be scaled by log10

    Returns:
    --------
    4 Figures of 2D heatmap for each criteria
    '''
    # set heatmap x,y ranges
    xLabel = x_range
    yLabel = y_range

    # D-optimality
    fig = plt.figure()
    plt.rc('axes', titlesize=font_axes)
    plt.rc('axes', labelsize=font_axes)
    plt.rc('xtick', labelsize=font_tick)
    plt.rc('ytick', labelsize=font_tick)
    ax = fig.add_subplot(111)
    params = {'mathtext.default': 'regular'}
    plt.rcParams.update(params)
    ax.set_yticks(range(len(yLabel)))
    ax.set_yticklabels(yLabel)
    ax.set_ylabel(ylabel_text)
    
    
    ax.set_xticks(range(len(xLabel)))
    ax.set_xticklabels(xLabel)
    loc = ticker.MultipleLocator(base=3)
    ax.xaxis.set_major_locator(loc)
    ax.set_xlabel(xlabel_text)

    im = ax.imshow(d_result, cmap=plt.cm.hot_r)
    ba = plt.colorbar(im)
    ba.set_label('$log_{10}$(A-optimality)',labelpad=-40, y=1.1, rotation=0)
    plt.show()
